# Classifying CIFAR 100 with a pretrained model

In [ ]:
import tensorflow as tf

# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar100.load_data(label_mode="coarse")

# Print the shape of the data
print(f'Training data shape: {x_train.shape}, Training labels shape: {y_train.shape}')
print(f'Test data shape: {x_test.shape}, Test labels shape: {y_test.shape}')
num_classes = len(set(y_train.flatten()))
print(f'Number of different labels: {num_classes}')

In [ ]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0


In [ ]:
from utils import plot_label_distribution
from sklearn.model_selection import train_test_split
import numpy as np

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

import matplotlib.pyplot as plt


# Plot label distribution for training, validation, and test sets
plot_label_distribution(y_train, 'Training Set Label Distribution')
plot_label_distribution(y_val, 'Validation Set Label Distribution')
plot_label_distribution(y_test, 'Test Set Label Distribution')
#
print(f'New training data shape: {x_train.shape}, New training labels shape: {y_train.shape}')
print(f'Validation data shape: {x_val.shape}, Validation labels shape: {y_val.shape}')
print(f'Test data shape: {x_val.shape}, Test labels shape: {y_val.shape}')


In [ ]:
train_dataset = (x_train, y_train)
val_dataset = (x_val, y_val)
test_dataset = (x_test, y_test)

## Try a pair of models using feature extraction

In [ ]:
import keras
conv_base_resnet = keras.applications.ResNet50(include_top = False,
                                        weights = 'imagenet',
                                        input_shape= (32,32,3))

In [ ]:
conv_base_resnet.trainable = False

In [ ]:
conv_base_resnet.summary()

In [ ]:
from keras import layers

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)
  
inputs = keras.Input(shape=(32, 32, 3))
x = data_augmentation(inputs)
x = conv_base_resnet(inputs)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(20, activation="softmax")(x)
model_resnet_fe = keras.Model(inputs, outputs)
model_resnet_fe.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="models/resnet_fe.keras",
        save_best_only=True,
        monitor="val_loss"),
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True)
]
history = model_resnet_fe.fit(
    x_train, y_train,
    epochs=200, 
    validation_data=(x_val, y_val), 
    callbacks=callbacks,
    batch_size=128) 

In [ ]:
from utils import plot 

plot(history)

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Evaluate the model on the test set
test_loss, test_accuracy = model_resnet_fe.evaluate(x_test, y_test, verbose=2)

print(f'Test accuracy: {test_accuracy * 100:.2f}%')
print(f'Test loss: {test_loss:.4f}')

# Predict the classes for the test set
y_pred = model_resnet_fe.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Define the category names
category_names = [
    "aquatic mammals",
    "fish",
    "flowers",
    "food containers",
    "fruit and vegetables",
    "household electrical devices",
    "household furniture",
    "insects",
    "large carnivores",
    "large man-made outdoor things",
    "large natural outdoor scenes",
    "large omnivores and herbivores",
    "medium-sized mammals",
    "non-insect invertebrates",
    "people",
    "reptiles",
    "small mammals",
    "trees",
    "vehicles 1",
    "vehicles 2"
]

# Plot the confusion matrix with category names
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=category_names, yticklabels=category_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
conv_base_efficient = keras.applications.EfficientNetB3(include_top = False,
                                        weights = 'imagenet',
                                        input_shape= (32,32,3))
conv_base_efficient.trainable = False
conv_base_efficient.summary()


In [ ]:
from keras import layers

data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)
  
inputs = keras.Input(shape=(32, 32, 3))
x = data_augmentation(inputs)
x = conv_base_efficient(inputs)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(20, activation="softmax")(x)
model_efficient_fe = keras.Model(inputs, outputs)
model_efficient_fe.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="models/efficientnet_fe.keras",
        save_best_only=True,
        monitor="val_loss"),
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True)
]
history = model_efficient_fe.fit(
    x_train, y_train,
    epochs=200, 
    validation_data=(x_val, y_val), 
    callbacks=callbacks,
    batch_size=128) 

In [ ]:
plot(history)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model_efficient_fe.evaluate(x_test, y_test, verbose=2)

print(f'Test accuracy: {test_accuracy * 100:.2f}%')
print(f'Test loss: {test_loss:.4f}')

# Predict the classes for the test set
y_pred = model_efficient_fe.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Plot the confusion matrix with category names
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=category_names, yticklabels=category_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

## Train the same models but using fine tuning 

In [ ]:
conv_base_resnet.trainable = True

for layer in conv_base_resnet.layers[:-3]:
    layer.trainable = False

conv_base_resnet.summary()

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)
  
inputs = keras.Input(shape=(32, 32, 3))
x = data_augmentation(inputs)
x = conv_base_resnet(inputs)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(20, activation="softmax")(x)
model_resnet_ft = keras.Model(inputs, outputs)
model_resnet_ft.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="models/resnet_ft.keras",
        save_best_only=True,
        monitor="val_loss"),
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True)
]
history = model_resnet_ft.fit(
    x_train, y_train,
    epochs=200, 
    validation_data=(x_val, y_val), 
    callbacks=callbacks,
    batch_size=128) 

In [ ]:
plot(history)

In [ ]:

# Evaluate the model on the test set
test_loss, test_accuracy = model_resnet_ft.evaluate(x_test, y_test, verbose=2)

print(f'Test accuracy: {test_accuracy * 100:.2f}%')
print(f'Test loss: {test_loss:.4f}')

# Predict the classes for the test set
y_pred = model_resnet_ft.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Plot the confusion matrix with category names
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=category_names, yticklabels=category_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
conv_base_efficient.trainable = True

for layer in conv_base_efficient.layers[:-5]:
    layer.trainable = False

conv_base_efficient.summary()

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2),
    ]
)
  
inputs = keras.Input(shape=(32, 32, 3))
x = data_augmentation(inputs)
x = conv_base_efficient(inputs)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(20, activation="softmax")(x)
model_efficient_ft = keras.Model(inputs, outputs)
model_efficient_ft.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="models/efficient_ft.keras",
        save_best_only=True,
        monitor="val_loss"),
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=10,
        restore_best_weights=True)
]
history = model_efficient_ft.fit(
    x_train, y_train,
    epochs=200, 
    validation_data=(x_val, y_val), 
    callbacks=callbacks,
    batch_size=128) 

In [ ]:
plot(history)

In [ ]:

from sklearn.metrics import confusion_matrix
import seaborn as sns

# Evaluate the model on the test set
test_loss, test_accuracy = model_efficient_ft.evaluate(x_test, y_test, verbose=2)

print(f'Test accuracy: {test_accuracy * 100:.2f}%')
print(f'Test loss: {test_loss:.4f}')

# Predict the classes for the test set
y_pred = model_efficient_ft.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Generate the confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Plot the confusion matrix with category names
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=category_names, yticklabels=category_names)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()